# <center> ZO-Ridge Regression
    

## Import libraries

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from models import linearReg
from simus import run_ols
import warnings
warnings.filterwarnings("ignore")
from tqdm.notebook import tqdm

## Generate Dataset for Regression

In [ ]:
def simu_block(seed,n_samples,n_features,puiss,block_size,noise):
    np.random.seed(seed)
    X = np.zeros((n_samples,n_features))
    for j in range(n_features//block_size):
        X_j = np.random.normal(scale=(j+1)**(-puiss),size=(n_samples,block_size))
        X[:,j*block_size:(j+1)*block_size] = X_j
    # shuffle columns of X
    idx = np.random.permutation(n_features)
    X[:, :] = X[:, idx]
    ground_truth = np.random.uniform(low=-1,high=1,size=n_features)
    y = X@ground_truth
    if noise > 0.0:
        y += np.random.normal(scale=noise, size=y.shape)
    return X, y, ground_truth

In [ ]:
n_samples = 10000
n_features = 250
# Simulate data for regression
seed=0
#uiss=5
puiss=10
noise=0.01
block_size=10

In [ ]:
X,y,coeff=simu_block(seed=seed,n_samples=n_samples,n_features=n_features,
                     puiss=puiss,block_size=block_size,noise=noise)

## Compute true solution

In [ ]:
𝜆 = 1/n_samples          #regularization parameter
G = ((X.T)@X)/n_samples  # Gram matrix
A = G + 𝜆*np.eye(n_features)
B = ((X.T)@y)/n_samples
ridge = np.linalg.solve(a=A ,b=B)

In [ ]:
data_opt = np.sum((y-X.dot(ridge))**2)/(2*n_samples)
reg_opt = (𝜆/2) * sum(ridge**2)
loss_opt = data_opt + reg_opt
print('data_opt:',data_opt)
print('reg_opt :',reg_opt)
print(loss_opt)

In [ ]:
ols = linearReg(X=X,y=y,𝜆=𝜆)

In [ ]:
ols.loss(batch=np.arange(n_samples),w=ridge)

In [ ]:
ols.loss(batch=np.arange(n_samples),w=np.zeros(n_features))

## Parameter Simulations

In [ ]:
N = int(200)  # number of passes over coordinates          
a = 1          # numerator of learning rate
t0 = 10
alpha_power = 1 # power in the learning rate
gamma = 1       # numerator in gradient factor smoothing
mu_power = 1    # power in the gradient factor smoothing
verbose = False # to display information
N_exp = 20     # number of experiments
fixed=False
eta = 0.5
#fixed = True
batch_size = 1
T = int(np.sqrt(n_features)) # size of exploration
print('T=  ',T)
print('𝜆=  ',𝜆)

print('T=  ',T)
print('𝜆=  ',𝜆)
print('eta=',eta)

# Run different ZO methods

## Full gradient estimate

In [ ]:
_,_,loss_full = run_ols(X=X,y=y,𝜆=𝜆,batch_size=batch_size,method='full',N_exp=N_exp,N=N,
                        T=None,gamma=gamma,mu_power=mu_power,a=a,t0=t0,alpha_power=alpha_power,
                        fixed=None,eta=None,importance=None,gains=None,verbose=False)
l_ful = np.mean(loss_full,axis=0)
std_ful = np.std(loss_full,axis=0)

## Uniform coordinate sampling

In [ ]:
_,_,loss_uni = run_ols(X=X,y=y,𝜆=𝜆,batch_size=batch_size,method='uni',N_exp=N_exp,N=N,
                       T=None,gamma=gamma,mu_power=mu_power,a=a,t0=t0,alpha_power=alpha_power,
                      fixed=None,eta=None,importance=None,gains=None,verbose=False)
l_uni = np.mean(loss_uni,axis=0)
std_uni = np.std(loss_uni,axis=0)

## Musketeer biased (with different gains: Average, Absolute Value, Square)

In [ ]:
_,_,loss_mus_avg = run_ols(X=X,y=y,𝜆=𝜆,batch_size=batch_size,method='mus',N_exp=N_exp,N=N,
                           T=T,gamma=gamma,mu_power=mu_power,a=a,t0=t0,alpha_power=alpha_power,
                           fixed=fixed,eta=eta,importance=False,gains='avg',
                           verbose=False)
l_avg = np.mean(loss_mus_avg,axis=0)
std_avg = np.std(loss_mus_avg,axis=0)

_,_,loss_mus_abs = run_ols(X=X,y=y,𝜆=𝜆,batch_size=batch_size,method='mus',N_exp=N_exp,N=N,
                           T=T,gamma=gamma,mu_power=mu_power,a=a,t0=t0,alpha_power=alpha_power,
                           fixed=fixed,eta=eta,importance=False,gains='abs',
                           verbose=False)
l_abs = np.mean(loss_mus_abs,axis=0)
std_abs = np.std(loss_mus_abs,axis=0)

_,_,loss_mus_sqr = run_ols(X=X,y=y,𝜆=𝜆,batch_size=batch_size,method='mus',N_exp=N_exp,N=N,
                           T=T,gamma=gamma,mu_power=mu_power,a=a,t0=t0,alpha_power=alpha_power,
                           fixed=fixed,eta=eta,importance=False,gains='square',
                           verbose=False)
l_sqr = np.mean(loss_mus_sqr,axis=0)
std_sqr = np.std(loss_mus_sqr,axis=0)

### Gaussian smoothing estimate (Nesterov-Spokoiny)

In [ ]:
_,_,loss_nes = run_ols(X=X,y=y,𝜆=𝜆,batch_size=batch_size,method='nes',N_exp=N_exp,N=N,
                       T=T,gamma=gamma,mu_power=mu_power,a=a,t0=t0,alpha_power=alpha_power,
                       fixed=fixed,eta=eta,importance=False,gains='avg',
                       verbose=False)
l_nes = np.mean(loss_nes,axis=0)
std_nes = np.std(loss_nes,axis=0)

## Save Results

In [ ]:
#np.save('loss_ful_ridge_puiss10.npy',loss_full)
#np.save('loss_uni_ridge_puiss10.npy',loss_uni)
#np.save('loss_avg_ridge_puiss10.npy',loss_mus_avg)
#np.save('loss_abs_ridge_puiss10.npy',loss_mus_abs)
#np.save('loss_sqr_ridge_puiss10.npy',loss_mus_sqr)
#np.save('loss_nes_ridge_puiss10.npy',loss_nes)

### Musketeer -Importance Sampling (IS)

In [ ]:
_,_,loss_mus_avg_is = run_ols(X=X,y=y,𝜆=𝜆,batch_size=batch_size,
                       method='mus',N_exp=N_exp,N=N,
                       T=T,gamma=gamma,mu_power=mu_power,a=a,alpha_power=alpha_power,
                       fixed=fixed,eta=eta,importance=True,gains='avg',
                       verbose=False)
l_avg_is = np.mean(loss_mus_avg_is,axis=0)


_,_,loss_mus_abs_is = run_ols(X=X,y=y,𝜆=𝜆,batch_size=batch_size,
                       method='mus',N_exp=N_exp,N=N,
                       T=T,gamma=gamma,mu_power=mu_power,a=a,alpha_power=alpha_power,
                       fixed=fixed,eta=eta,importance=True,gains='abs',
                       verbose=False)
l_abs_is = np.mean(loss_mus_abs_is,axis=0)


_,_,loss_mus_sqr_is = run_ols(X=X,y=y,𝜆=𝜆,batch_size=batch_size,
                       method='mus',N_exp=N_exp,N=N,
                       T=T,gamma=gamma,mu_power=mu_power,a=a,alpha_power=alpha_power,
                       fixed=fixed,eta=eta,importance=True,gains='square',
                       verbose=False)
l_sqr_is = np.mean(loss_mus_sqr_is,axis=0)

# Run experiments in different settings (Appendix E)

In [ ]:
N = int(200)  # number of passes over coordinates          
a = 1          # numerator of learning rate
t0 = 10
alpha_power = 1 # power in the learning rate
gamma = 1       # numerator in gradient factor smoothing
mu_power = 1    # power in the gradient factor smoothing
verbose = False # to display information
N_exp = 20     # number of experiments
fixed=False
eta = 0.5
batch_size = 1

seed=0
puiss=5
noise=0.01
block_size=10
for n_samples in [1000,2000,5000]:
    for n_features in [20,50,100,200]:
        print('n = ',n_samples)
        print('p = ',n_features)
        T = int(np.sqrt(n_features)) # size of exploration
        # Generate data for regression
        X,y,coeff=simu_block(seed=seed,n_samples=n_samples,n_features=n_features,
                     puiss=puiss,block_size=block_size,noise=noise)
        # Compute true solution
        𝜆 = 1/n_samples          #regularization parameter
        G = ((X.T)@X)/n_samples  # Gram matrix
        A = G + 𝜆*np.eye(n_features)
        B = ((X.T)@y)/n_samples
        ridge = np.linalg.solve(a=A ,b=B)
        data_opt = np.sum((y-X.dot(ridge))**2)/(2*n_samples)
        reg_opt = (𝜆/2) * sum(ridge**2)
        loss_opt = data_opt + reg_opt
        # Run simulation with different ZO methods for regression
        _,_,loss_full = run_ols(X=X,y=y,𝜆=𝜆,batch_size=batch_size,method='full',N_exp=N_exp,N=N,
                        T=None,gamma=gamma,mu_power=mu_power,a=a,t0=t0,alpha_power=alpha_power,
                        fixed=None,eta=None,importance=None,gains=None,verbose=False)
        _,_,loss_uni = run_ols(X=X,y=y,𝜆=𝜆,batch_size=batch_size,method='uni',N_exp=N_exp,N=N,
                       T=None,gamma=gamma,mu_power=mu_power,a=a,t0=t0,alpha_power=alpha_power,
                      fixed=None,eta=None,importance=None,gains=None,verbose=False)
        _,_,loss_avg = run_ols(X=X,y=y,𝜆=𝜆,batch_size=batch_size,method='mus',N_exp=N_exp,N=N,
                           T=T,gamma=gamma,mu_power=mu_power,a=a,t0=t0,alpha_power=alpha_power,
                           fixed=fixed,eta=eta,importance=True,gains='avg',
                           verbose=False)
        _,_,loss_abs = run_ols(X=X,y=y,𝜆=𝜆,batch_size=batch_size,method='mus',N_exp=N_exp,N=N,
                                   T=T,gamma=gamma,mu_power=mu_power,a=a,t0=t0,alpha_power=alpha_power,
                                   fixed=fixed,eta=eta,importance=True,gains='abs',
                                   verbose=False)
        _,_,loss_sqr = run_ols(X=X,y=y,𝜆=𝜆,batch_size=batch_size,method='mus',N_exp=N_exp,N=N,
                                   T=T,gamma=gamma,mu_power=mu_power,a=a,t0=t0,alpha_power=alpha_power,
                                   fixed=fixed,eta=eta,importance=True,gains='square',
                                   verbose=False)
        _,_,loss_nes = run_ols(X=X,y=y,𝜆=𝜆,batch_size=batch_size,method='nes',N_exp=N_exp,N=N,
                       T=T,gamma=gamma,mu_power=mu_power,a=a,t0=t0,alpha_power=alpha_power,
                       fixed=fixed,eta=eta,importance=False,gains='avg',
                       verbose=False)
        #np.save('loss_ful_ridge_n{}_p{}.npy'.format(n_samples,n_features),loss_full-loss_opt)
        #np.save('loss_uni_ridge_n{}_p{}.npy'.format(n_samples,n_features),loss_uni-loss_opt)
        #np.save('loss_nes_ridge_n{}_p{}.npy'.format(n_samples,n_features),loss_nes-loss_opt)
        #np.save('loss_avg_is_ridge_n{}_p{}.npy'.format(n_samples,n_features),loss_avg-loss_opt)
        #np.save('loss_sqr_is_ridge_n{}_p{}.npy'.format(n_samples,n_features),loss_sqr-loss_opt)
        #np.save('loss_abs_is_ridge_n{}_p{}.npy'.format(n_samples,n_features),loss_abs-loss_opt)